In [3]:
import pandas as pd


In [4]:
df = pd.read_csv("C:\\Users\\Aman\\Desktop\\kifyaw6\\data\\data.csv")

In [5]:
df.columns

Index(['TransactionId', 'BatchId', 'AccountId', 'SubscriptionId', 'CustomerId',
       'CurrencyCode', 'CountryCode', 'ProviderId', 'ProductId',
       'ProductCategory', 'ChannelId', 'Amount', 'Value',
       'TransactionStartTime', 'PricingStrategy', 'FraudResult'],
      dtype='object')